# Usecase Delivery Planning Agent

This notebook implements a RAG-based delivery planning agent using DSPy that helps users generate comprehensive project plans for data migration and delivery projects. The agent leverages indexed documents and follows a structured approach to gather requirements and generate actionable project plans.

## Features:
- **Intelligent Question Generation**: Automatically generates relevant questions based on project context
- **Document Retrieval**: Uses vector search to find relevant information from indexed documents
- **Structured Planning**: Generates comprehensive project plans with timelines, resources, and milestones
- **Risk Assessment**: Identifies potential risks and mitigation strategies
- **DSPy Optimization**: Uses DSPy for prompt optimization and performance improvement


## 1. Setup and Configuration


In [0]:
# Install required packages
# Note: These versions are compatible with Databricks Runtime 16.4 LTS
%pip install dspy-ai databricks-vectorsearch pydantic databricks-agents

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# Restart Python to ensure packages are loaded
dbutils.library.restartPython()


In [0]:
# Configuration
VECTOR_SEARCH_ENDPOINT_NAME = "usecase-agent" #"use-case-planning-agent"  # Your vector search endpoint
VECTOR_SEARCH_INDEX_NAME = "vbdemos.usecase_agent.migration_plan_pdfs" #"vbdemos.usecase_agent.migration_plan_pdfs"  # Your existing vector search index
CATALOG_NAME = "vbdemos"
SCHEMA_NAME = "usecase_agent"
DOCUMENTS_TABLE = f"{CATALOG_NAME}.{SCHEMA_NAME}.usecase_planning_agent_pdf_parsed"  # Source table
VECTOR_INDEX_TABLE = f"{CATALOG_NAME}.{SCHEMA_NAME}.migration_plan_pdfs"  # Vector search index table

# Project planning categories and questions
PLANNING_CATEGORIES = {
    "Resource": [
        "How many team members are there?",
        "Are they using PS skills?",
        "Are they using an SI?",
        "Are the teams sufficiently skilled/trained?",
        "Are resources shared with other projects?",
        "Have resources done this work before?",
        "Is there a product owner?",
        "Are the DevOps/SecOps/Infra teams under this 'teams' control/purview?",
        "Program manager?",
        "Are the BAU teams that will ultimately manage the new system sufficiently trained?",
        "Has an end-user adoption plan been created?"
    ],
    "Current Process Maturity": [
        "Do they have a history of delays?",
        "Do they have change management authority/equivalent?",
        "Do they have an identified way of working - agile/waterfall?"
    ],
    "Customer Background": [
        "Does the customer have a specific deadline/reason for that deadline?",
        "Customer is already using cloud?",
        "Customer has Databricks elsewhere?",
        "Customer has security approval for this migration?",
        "Are there any key connectors that are needed?",
        "What are the key drivers of the migration?",
        "Are there any legal compliance or requirements to consider?"
    ],
    "Scope": [
        "Has a pilot or POC been conducted?",
        "Does the customer have visibility of all the data and pipelines that need migration?",
        "Is customer aware of where and who uses the data?",
        "Is lift and shift or redesign preferred?",
        "How many pipelines are to be migrated?",
        "Relative complexity of pipelines?",
        "Volume of data to be migrated?",
        "How frequently is the data updated?",
        "Is there a proposed UC design/infrastructure design?",
        "Is PII handling included?",
        "Does the migration include monitoring?",
        "Does the migration include optimization?",
        "Will it be run in parallel or phased move over?",
        "Awareness of business critical pipelines/pipelines that cannot be down?",
        "Do they have control over how they receive data?",
        "Are additional data quality checks needing to be implemented?",
        "Are there any key connectors that need to be migrated?",
        "What level of testing is required and who will be doing this?",
        "Are data consumers/systems that use the data known?",
        "Does customer already have a plan?",
        "What is the quality of the data?",
        "Are the data pathways known?",
        "Has a permissions model been agreed?",
        "Is there a new data layout/has the UC catalog/schema structure been designed and agreed?",
        "Is disaster recovery included in this migration?"
    ]
}

print("Configuration loaded successfully!")
print(f"Vector Search Endpoint: {VECTOR_SEARCH_ENDPOINT_NAME}")
print(f"Vector Search Index: {VECTOR_SEARCH_INDEX_NAME}")
print(f"Documents Table: {DOCUMENTS_TABLE}")
print(f"Vector Index Table: {VECTOR_INDEX_TABLE}")
print(f"Planning Categories: {list(PLANNING_CATEGORIES.keys())}")
print("\nNote: Vector search index must be in format <catalog>.<schema>.<table>")
print("Vector index structure: path (string), text (string), __db_text_vector (array)")


Configuration loaded successfully!
Vector Search Endpoint: usecase-agent
Vector Search Index: vbdemos.usecase_agent.migration_plan_pdfs
Documents Table: vbdemos.usecase_agent.usecase_planning_agent_pdf_parsed
Vector Index Table: vbdemos.usecase_agent.migration_plan_pdfs
Planning Categories: ['Resource', 'Current Process Maturity', 'Customer Background', 'Scope']

Note: Vector search index must be in format <catalog>.<schema>.<table>
Vector index structure: path (string), text (string), __db_text_vector (array)


## 2. DSPy Setup and Model Configuration


In [0]:
import dspy
import json
from typing import List, Dict, Any
from databricks.vector_search.client import VectorSearchClient
from pyspark.sql import SparkSession

# Initialize Spark
spark = SparkSession.builder.appName("DeliveryPlanningAgent").getOrCreate()

# Configure DSPy with Databricks model
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
url = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get() + '/serving-endpoints'

lm = dspy.LM(
    model="databricks/databricks-claude-sonnet-4",
    api_key=token,
    api_base=url,
)

# Configure DSPy with Databricks model
# Note: MLflow integration is handled automatically in Databricks Runtime 16.4 LTS
# The warnings about MLflowCallback can be safely ignored as they don't affect functionality
dspy.configure(lm=lm)

print(f"DSPy version: {dspy.__version__}")
print("DSPy configured successfully with Databricks model")

# Initialize Vector Search client
vsc = VectorSearchClient(disable_notice=True)

print("DSPy and Vector Search configured successfully!")


DSPy version: 3.0.3
DSPy configured successfully with Databricks model
DSPy and Vector Search configured successfully!


## 3. DSPy Signatures and Modules


In [0]:
# DSPy Signatures for the delivery planning agent

class QuestionGenerator(dspy.Signature):
    """Generate relevant questions for migrating TO Databricks from existing data/analytics platforms."""
    project_context: str = dspy.InputField(desc="Description of the current data/analytics platform and objectives for migrating TO Databricks")
    category: str = dspy.InputField(desc="Planning category (Resource, Scope, Customer Background, etc.)")
    existing_answers: str = dspy.InputField(desc="Previously answered questions and responses")
    questions: str = dspy.OutputField(desc="List of 3-5 most relevant questions for this category specific to migrating TO Databricks")

class DocumentRetriever(dspy.Signature):
    """Retrieve relevant documents based on questions and context."""
    question: str = dspy.InputField(desc="Specific question to find relevant information for")
    project_context: str = dspy.InputField(desc="Project context and background")
    retrieved_docs: str = dspy.OutputField(desc="Relevant document excerpts and information")

class AnswerAnalyzer(dspy.Signature):
    """Analyze answers and extract key insights for project planning."""
    question: str = dspy.InputField(desc="The question that was asked")
    answer: str = dspy.InputField(desc="The answer provided by the user")
    relevant_docs: str = dspy.InputField(desc="Relevant documents retrieved from knowledge base")
    insights: str = dspy.OutputField(desc="Key insights, risks, and implications for project planning")

class ProjectPlanGenerator(dspy.Signature):
    """Generate comprehensive migration plan for moving TO Databricks with structured table outputs."""
    project_context: str = dspy.InputField(desc="Overall project context and objectives for migrating TO Databricks")
    gathered_insights: str = dspy.InputField(desc="All insights gathered from questions and documents about current platform")
    timeline_requirements: str = dspy.InputField(desc="Timeline constraints and requirements for migrating TO Databricks")
    project_plan: str = dspy.OutputField(desc="Comprehensive migration plan for moving TO Databricks. Output as structured tables with clear headers for: 1) Migration Timeline, 2) Resource Requirements, 3) Migration Phases, 4) Risk Assessment. Use markdown table format with | separators.")

class RiskAssessor(dspy.Signature):
    """Assess risks and provide mitigation strategies."""
    project_plan: str = dspy.InputField(desc="The proposed project plan")
    project_context: str = dspy.InputField(desc="Project context and constraints")
    risk_assessment: str = dspy.OutputField(desc="Identified risks, their likelihood, impact, and mitigation strategies")

print("DSPy signatures defined successfully!")


DSPy signatures defined successfully!


In [0]:
# DSPy Modules for the delivery planning agent

class QuestionGenerationModule(dspy.Module):
    """Module to generate relevant questions for each planning category."""
    
    def __init__(self):
        super().__init__()
        self.generate = dspy.ChainOfThought(QuestionGenerator)
    
    def forward(self, project_context: str, category: str, existing_answers: str = ""):
        return self.generate(
            project_context=project_context,
            category=category,
            existing_answers=existing_answers
        )

class DocumentRetrievalModule(dspy.Module):
    """Module to retrieve relevant documents using vector search."""
    
    def __init__(self, vector_search_endpoint: str, vector_search_index: str):
        super().__init__()
        self.vector_search_endpoint = vector_search_endpoint
        self.vector_search_index = vector_search_index
        self.retrieve = dspy.ChainOfThought(DocumentRetriever)
    
    def forward(self, question: str, project_context: str):
        # Perform vector search
        search_results = vsc.get_index(
            endpoint_name=self.vector_search_endpoint,
            index_name=self.vector_search_index
        ).similarity_search(
            query_text=question,
            columns=["path", "text"],  # Updated to match your vector index structure,
            num_results=5
        )
        
        # Format retrieved documents
        # Handle the case where search_results might be a list or dict
        if isinstance(search_results, list):
            documents = search_results
        else:
            documents = search_results.get('result', {}).get('data_array', [])
        
        retrieved_docs = "\n\n".join([
            f"Source: {doc.get('path', 'Unknown') if isinstance(doc, dict) else 'Unknown'}\nContent: {doc.get('text', '') if isinstance(doc, dict) else str(doc)}"
            for doc in documents
        ])
        
        return self.retrieve(
            question=question,
            project_context=project_context,
            retrieved_docs=retrieved_docs
        )

class AnswerAnalysisModule(dspy.Module):
    """Module to analyze answers and extract insights."""
    
    def __init__(self):
        super().__init__()
        self.analyze = dspy.ChainOfThought(AnswerAnalyzer)
    
    def forward(self, question: str, answer: str, relevant_docs: str):
        return self.analyze(
            question=question,
            answer=answer,
            relevant_docs=relevant_docs
        )

class ProjectPlanGenerationModule(dspy.Module):
    """Module to generate comprehensive project plans."""
    
    def __init__(self):
        super().__init__()
        self.generate_plan = dspy.ChainOfThought(ProjectPlanGenerator)
    
    def forward(self, project_context: str, gathered_insights: str, timeline_requirements: str):
        return self.generate_plan(
            project_context=project_context,
            gathered_insights=gathered_insights,
            timeline_requirements=timeline_requirements
        )

class RiskAssessmentModule(dspy.Module):
    """Module to assess risks and provide mitigation strategies."""
    
    def __init__(self):
        super().__init__()
        self.assess_risks = dspy.ChainOfThought(RiskAssessor)
    
    def forward(self, project_plan: str, project_context: str):
        return self.assess_risks(
            project_plan=project_plan,
            project_context=project_context
        )

print("DSPy modules defined successfully!")


DSPy modules defined successfully!


## 4. Main Delivery Planning Agent


In [0]:
class DeliveryPlanningAgent(dspy.Module):
    """Main delivery planning agent that orchestrates the entire planning process."""
    
    def __init__(self, vector_search_endpoint: str, vector_search_index: str):
        super().__init__()
        self.question_generator = QuestionGenerationModule()
        self.document_retriever = DocumentRetrievalModule(vector_search_endpoint, vector_search_index)
        self.answer_analyzer = AnswerAnalysisModule()
        self.plan_generator = ProjectPlanGenerationModule()
        self.risk_assessor = RiskAssessmentModule()
        
        # Store conversation state
        self.conversation_history = []
        self.gathered_insights = []
        self.project_context = ""
    
    def start_planning_session(self, project_context: str, timeline_requirements: str = ""):
        """Start a new planning session."""
        self.project_context = project_context
        self.conversation_history = []
        self.gathered_insights = []
        
        print(" Starting Delivery Planning Session")
        print(f" Project Context: {project_context}")
        if timeline_requirements:
            print(f"⏰ Timeline Requirements: {timeline_requirements}")
        print("\n" + "="*50)
        
        return self._generate_questions_for_category("Resource")
    
    def _generate_questions_for_category(self, category: str):
        """Generate questions for a specific category."""
        existing_answers = self._format_existing_answers()
        
        result = self.question_generator(
            project_context=self.project_context,
            category=category,
            existing_answers=existing_answers
        )
        
        print(f"\n {category} Questions:")
        print("-" * 30)
        questions = result.questions.split('\n')
        for i, question in enumerate(questions, 1):
            if question.strip():
                print(f"{i}. {question.strip()}")
        
        return questions
    
    def answer_question(self, question: str, answer: str, category: str = ""):
        """Process a user's answer to a question."""
        print(f"\n Processing Answer:")
        print(f"Q: {question}")
        print(f"A: {answer}")
        
        # Retrieve relevant documents
        doc_result = self.document_retriever(question, self.project_context)
        relevant_docs = doc_result.retrieved_docs
        
        # Analyze the answer
        analysis_result = self.answer_analyzer(question, answer, relevant_docs)
        insights = analysis_result.insights
        
        # Store in conversation history
        self.conversation_history.append({
            "question": question,
            "answer": answer,
            "category": category,
            "insights": insights,
            "relevant_docs": relevant_docs
        })
        
        self.gathered_insights.append(insights)
        
        print(f"\n Key Insights:")
        print(insights)
        
        return insights
    
    def generate_project_plan(self, timeline_requirements: str = ""):
        """Generate the final project plan based on all gathered information."""
        print("\n Generating Project Plan...")
        print("="*50)
        
        # Combine all insights
        all_insights = "\n\n".join(self.gathered_insights)
        
        # Generate project plan
        plan_result = self.plan_generator(
            project_context=self.project_context,
            gathered_insights=all_insights,
            timeline_requirements=timeline_requirements
        )
        
        project_plan = plan_result.project_plan
        
        print(" PROJECT PLAN:")
        print("="*50)
        print(project_plan)
        
        # Assess risks
        risk_result = self.risk_assessor(project_plan, self.project_context)
        risk_assessment = risk_result.risk_assessment
        
        print("\n  RISK ASSESSMENT:")
        print("="*50)
        print(risk_assessment)
        
        return {
            "project_plan": project_plan,
            "risk_assessment": risk_assessment,
            "conversation_history": self.conversation_history
        }
    
    def _format_existing_answers(self):
        """Format existing answers for context."""
        if not self.conversation_history:
            return "No previous answers yet."
        
        formatted = []
        for entry in self.conversation_history:
            formatted.append(f"Q: {entry['question']}\nA: {entry['answer']}")
        
        return "\n\n".join(formatted)
    
    def get_next_category_questions(self):
        """Get questions for the next planning category."""
        categories = list(PLANNING_CATEGORIES.keys())
        completed_categories = set(entry.get('category', '') for entry in self.conversation_history)
        
        for category in categories:
            if category not in completed_categories:
                return self._generate_questions_for_category(category)
        
        return None  # All categories completed

print("Delivery Planning Agent defined successfully!")


Delivery Planning Agent defined successfully!


## 5. Example Usage and Testing


In [0]:
# Initialize the delivery planning agent
agent = DeliveryPlanningAgent(VECTOR_SEARCH_ENDPOINT_NAME, VECTOR_SEARCH_INDEX_NAME)

# Example: Start a planning session
project_context = """
We are planning a data migration project for a large financial services company. 
They want to migrate their existing Oracle data warehouse to Databricks on Azure. 
The migration involves 50+ data pipelines, 2TB of data, and needs to be completed 
within 6 months. The company has some cloud experience but limited Databricks knowledge.
"""

timeline_requirements = "6 months deadline, must be completed by end of Q2 2024"

# Start the planning session
questions = agent.start_planning_session(project_context, timeline_requirements)

print("\n" + "="*50)
print("Example: Answer some questions to see the agent in action")
print("="*50)


 Starting Delivery Planning Session
 Project Context: 
We are planning a data migration project for a large financial services company. 
They want to migrate their existing Oracle data warehouse to Databricks on Azure. 
The migration involves 50+ data pipelines, 2TB of data, and needs to be completed 
within 6 months. The company has some cloud experience but limited Databricks knowledge.

⏰ Timeline Requirements: 6 months deadline, must be completed by end of Q2 2024


 Resource Questions:
------------------------------
1. 1. What is the current size and composition of your data engineering team, and how many team members can be dedicated to this migration project full-time?
3. 2. Do you have existing Azure cloud infrastructure and Databricks workspace provisioned, or will this need to be set up as part of the project scope?
5. 3. What is your allocated budget for this migration project, including licensing, training, external consulting, and potential infrastructure costs?
7. 4. Are 

Trace(trace_id=tr-6cd632e54663cc3a2688fedf541a9ae0)

In [0]:
# Example: Simulate answering questions
# In a real scenario, these would be user inputs

# Answer Resource questions
resource_answers = [
    ("How many team members are there?", "We have 8 team members including 2 data engineers, 3 analysts, 2 developers, and 1 project manager"),
    ("Are the teams sufficiently skilled/trained?", "The data engineers have some cloud experience but no Databricks experience. The analysts are familiar with SQL but not Spark. We need training."),
    ("Is there a product owner?", "Yes, we have a product owner from the business side who understands the data requirements"),
    ("Are the DevOps/SecOps/Infra teams under this 'teams' control/purview?", "No, they are separate teams with their own priorities and reporting lines")
]

print(" Processing Resource Questions...")
for question, answer in resource_answers:
    agent.answer_question(question, answer, "Resource")

print("\n" + "="*50)
print("Moving to next category...")
print("="*50)

# Get next category questions
next_questions = agent.get_next_category_questions()


 Processing Resource Questions...

 Processing Answer:
Q: How many team members are there?
A: We have 8 team members including 2 data engineers, 3 analysts, 2 developers, and 1 project manager
[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


2025/09/16 09:26:57 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content="[[ ## re...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])"



 Key Insights:
Key insights for project planning:
- **Team Size**: 8-member team provides sufficient capacity for a complex data migration project
- **Role Distribution**: Good balance with emphasis on analysis (3 analysts) which is critical for data validation and testing phases
- **Technical Capacity**: 4 technical resources (2 data engineers + 2 developers) should handle pipeline development and migration tasks
- **Project Management**: Single PM may need support given project complexity (50+ pipelines, 2TB data, 6-month timeline)
- **Potential Risks**: Limited Databricks expertise mentioned in project context may require additional training or external support
- **Resource Allocation**: Consider if current staffing levels are adequate for 6-month timeline with 50+ pipeline migrations
- **Skill Gap Consideration**: May need to assess team's Databricks proficiency and plan for knowledge transfer or training

 Processing Answer:
Q: Are the teams sufficiently skilled/trained?
A: The d

2025/09/16 09:27:10 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## re...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])"
2025/09/16 09:27:17 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message


 Key Insights:
**Critical Skill Gap Identified**: Major deficiency in Databricks and Spark expertise creates high project risk. **Immediate Training Required**: Comprehensive upskilling program needed before migration begins - estimate 2-3 months for basic competency. **Timeline Impact**: Current 6-month timeline likely unrealistic without external expertise or extended training period. **Risk Mitigation Options**: (1) Engage Databricks professional services/certified partners, (2) Hire experienced Databricks developers, (3) Implement phased approach with extensive training, (4) Consider proof-of-concept to build skills before full migration. **Budget Implications**: Additional costs for training, certifications, external consultants, or new hires must be factored into project budget. **Success Factors**: Project success heavily dependent on either rapid skill acquisition or supplementing team with external Databricks expertise. Recommend immediate action on training plan and consider

...

2025/09/16 09:27:41 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## re...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])"
2025/09/16 09:27:52 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message


 Key Insights:
**Critical Risk Identified**: Organizational silos present major project delivery risk. Key implications:

**Dependencies & Bottlenecks**: Project success depends on teams outside direct control, creating potential scheduling conflicts and resource availability issues during critical migration phases.

**Governance Gap**: Need to establish formal escalation paths and service level agreements with DevOps/SecOps/Infra teams to ensure project requirements are prioritized appropriately.

**Timeline Risk**: 6-month timeline may be jeopardized if these teams have competing priorities or insufficient capacity to support migration activities.

**Mitigation Strategies Required**:
- Engage these teams early in project planning to secure commitments
- Establish clear communication channels and regular touchpoints
- Consider involving executive sponsors to ensure cross-team alignment
- Build buffer time into project schedule for potential delays from dependencies
- Document all req

2025/09/16 09:28:00 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## re...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])"



 Current Process Maturity Questions:
------------------------------
1. 1. What is your current data development lifecycle process (development, testing, deployment) and do you have established CI/CD practices for your Oracle data warehouse pipelines?
3. 2. How do you currently handle data quality monitoring, validation, and error handling in your existing Oracle environment?
5. 3. What data governance processes do you have in place today (data lineage tracking, metadata management, access controls, compliance procedures)?
7. 4. How do you currently manage environment promotion (dev/test/prod) and what change management processes exist for data pipeline modifications?
9. 5. What monitoring and alerting capabilities do you have for your current data pipelines, and how do you handle incident response and troubleshooting?


[Trace(trace_id=tr-25e2aabcd20e7a2f9302e96769d9bbdd), Trace(trace_id=tr-4e0683d2706ad1382ad4d88ae59c02eb), Trace(trace_id=tr-529a0848574e61ac1e32f422f03028e3), Trace(trace_id=tr-1b3f3d61260690c59fd5473424f530b5), Trace(trace_id=tr-9539abff3de554682630498b7c491595), Trace(trace_id=tr-4e2e741b207babc1ebb0d0b7448b56d0), Trace(trace_id=tr-9e55c39414ba18d6553bb006d7697309), Trace(trace_id=tr-b5f7c79b0d26dc353223034679bb90a2), Trace(trace_id=tr-850f5deb8ac95278cc2cc7b7fc8f62da)]

In [0]:
# Answer Scope questions
scope_answers = [
    ("How many pipelines are to be migrated?", "We have approximately 50 data pipelines that need to be migrated"),
    ("Volume of data to be migrated?", "The total data volume is about 2TB across multiple tables and schemas"),
    ("Is lift and shift or redesign preferred?", "We prefer a hybrid approach - lift and shift for simple pipelines, redesign for complex ones"),
    ("Does the migration include monitoring?", "Yes, we need comprehensive monitoring and alerting for the new system"),
    ("Will it be run in parallel or phased move over?", "We plan to do a phased migration over 6 months, starting with non-critical pipelines")
]

print(" Processing Scope Questions...")
for question, answer in scope_answers:
    agent.answer_question(question, answer, "Scope")

print("\n" + "="*50)
print("Generating Project Plan...")
print("="*50)

# Generate the final project plan
final_plan = agent.generate_project_plan(timeline_requirements)

print("\n Planning session completed!")
print(" Summary:")
print(f"- Total questions answered: {len(agent.conversation_history)}")
print(f"- Categories covered: {set(entry['category'] for entry in agent.conversation_history)}")
print(f"- Key insights gathered: {len(agent.gathered_insights)}")


 Processing Scope Questions...

 Processing Answer:
Q: How many pipelines are to be migrated?
A: We have approximately 50 data pipelines that need to be migrated
[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


2025/09/16 09:28:08 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## re...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])"
2025/09/16 09:28:16 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message


 Key Insights:
Key insights for project planning:
- **Scale Impact**: 50 pipelines represent a significant migration workload requiring systematic approach and phased execution
- **Resource Planning**: Will need dedicated teams for pipeline analysis, conversion, testing, and validation across all 50 pipelines
- **Timeline Considerations**: With 6-month deadline, this averages to ~8-9 pipelines per month, requiring parallel processing and efficient workflow management
- **Risk Factors**: High volume increases complexity and potential for cascading failures; need robust testing strategy for each pipeline
- **Dependencies**: Pipeline interdependencies must be mapped to determine migration sequence and avoid downstream impacts
- **Quality Assurance**: Each pipeline will require individual validation, data reconciliation, and performance testing
- **Change Management**: Significant scope affecting multiple business processes and stakeholders across the organization

 Processing Answer:
Q: 

2025/09/16 09:28:24 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## re...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])"
2025/09/16 09:28:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message


 Key Insights:
**Key Planning Insights:**
- **Migration Approach**: 2TB allows for both full and incremental migration strategies; recommend phased approach to minimize business disruption
- **Timeline Impact**: With 6-month constraint, allocate 2-3 months for data migration including testing phases
- **Resource Requirements**: Moderate volume requires dedicated migration tools (Azure Data Factory, Databricks Auto Loader) and sufficient network bandwidth (minimum 1Gbps recommended)
- **Risk Considerations**: Data validation becomes critical with this volume - implement automated data quality checks and reconciliation processes
- **Infrastructure Planning**: Ensure adequate Azure storage provisioning and Databricks cluster sizing to handle 2TB efficiently
- **Testing Strategy**: Plan for parallel testing environments to validate data integrity without impacting migration timeline
- **Rollback Planning**: 2TB volume requires robust backup and rollback procedures in case of migration iss

...

2025/09/16 09:30:15 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content="[[ ## re...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])"


 PROJECT PLAN:
## Migration Timeline

| Phase | Duration | Timeline | Key Activities | Success Criteria |
|-------|----------|----------|----------------|------------------|
| **Phase 0: Foundation** | 4 weeks | Weeks 1-4 | Team training, tool setup, external expertise onboarding, cross-team governance establishment | Databricks environment ready, team certified, governance framework active |
| **Phase 1: Pilot Migration** | 6 weeks | Weeks 5-10 | Migrate 5-8 non-critical pipelines, establish monitoring, validate processes | Successful migration of pilot pipelines, monitoring operational, processes validated |
| **Phase 2: Critical Systems** | 8 weeks | Weeks 11-18 | Migrate 20-25 critical pipelines, implement compliance controls, performance optimization | Critical business functions operational, compliance validated, performance benchmarks met |
| **Phase 3: Remaining Systems** | 6 weeks | Weeks 19-24 | Complete remaining 15-20 pipelines, full data validation, cutover preparation | A

2025/09/16 09:30:36 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## re...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])"



  RISK ASSESSMENT:
**CRITICAL RISKS (Immediate Action Required):**

1. **Regulatory Compliance Risk - VERY HIGH**
   - **Likelihood:** High | **Impact:** Severe
   - **Details:** Financial services face stringent regulations (SOX, Basel III, GDPR, etc.). Data lineage, audit trails, and compliance controls must be maintained throughout migration.
   - **Mitigation:** Engage compliance team from Week 1, implement comprehensive audit logging, establish regulatory sandbox environment, obtain pre-approval for architecture changes.
   - **Contingency:** Dedicated compliance workstream, external regulatory consultant, extended parallel operations until full compliance validation.

2. **Data Integrity & Business Continuity Risk - VERY HIGH**
   - **Likelihood:** Medium-High | **Impact:** Severe
   - **Details:** 2TB of financial data with 50+ pipelines creates multiple failure points. Any data corruption or loss could impact trading, reporting, and regulatory submissions.
   - **Mitigation:**

[Trace(trace_id=tr-c8208a074b95786d88e38d75ea2db04c), Trace(trace_id=tr-f1d6ecc0fb4a42050c7e30b698877d5c), Trace(trace_id=tr-f18c11c9c2cf3bbdf9253691403ab16f), Trace(trace_id=tr-afc216375233aaac4cceebda14bc5314), Trace(trace_id=tr-1954b394b5b407196857865a9a1eb2f4), Trace(trace_id=tr-c0ac3a7d6449aa9123c17f3997989fe8), Trace(trace_id=tr-6981523c71805f3a88e17d8126a84f1c), Trace(trace_id=tr-c7e29f784cf826c314fdd6914ce37bfe), Trace(trace_id=tr-12fbf73634c5ab7be0c038a83ec8703d), Trace(trace_id=tr-170493683ae15c873e27a32554f76302)]

## 6. Interactive Planning Interface


In [0]:
def interactive_planning_session():
    """Interactive function for users to run their own planning sessions."""
    
    print(" Welcome to the Delivery Planning Agent!")
    print("="*50)
    
    # Get project context from user
    project_context = input("Please describe your project context: ")
    timeline_requirements = input("Any specific timeline requirements? (press Enter to skip): ")
    
    # Initialize agent
    agent = DeliveryPlanningAgent(VECTOR_SEARCH_ENDPOINT_NAME, VECTOR_SEARCH_INDEX_NAME)
    
    # Start planning session
    questions = agent.start_planning_session(project_context, timeline_requirements)
    
    print("\n Please answer the questions above. Type 'next' to move to the next category, or 'plan' to generate the project plan.")
    
    while True:
        user_input = input("\nYour response (or 'next'/'plan'): ").strip()
        
        if user_input.lower() == 'next':
            next_questions = agent.get_next_category_questions()
            if next_questions is None:
                print(" All categories completed! Type 'plan' to generate your project plan.")
            continue
        
        elif user_input.lower() == 'plan':
            final_plan = agent.generate_project_plan(timeline_requirements)
            break
        
        else:
            # This is a simplified version - in practice, you'd need to match the response to the current question
            print("Please provide a more specific answer or use 'next'/'plan' commands.")
    
    return final_plan

# Uncomment the line below to run an interactive session
# interactive_planning_session()

print("Interactive planning interface ready!")
print("To start an interactive session, uncomment and run: interactive_planning_session()")


Interactive planning interface ready!
To start an interactive session, uncomment and run: interactive_planning_session()


In [0]:
interactive_planning_session()

 Welcome to the Delivery Planning Agent!


Please describe your project context:  CENTRICA ARE doing a hive to uc migration

Any specific timeline requirements? (press Enter to skip):  6 months

 Starting Delivery Planning Session
 Project Context: CENTRICA ARE doing a hive to uc migration
⏰ Timeline Requirements: 6 months



2025/09/16 09:36:08 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## re...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])"


...

 PROJECT PLAN:
## Migration Timeline

| Phase | Duration | Start Week | End Week | Key Deliverables |
|-------|----------|------------|----------|------------------|
| Assessment & Planning | 4 weeks | Week 1 | Week 4 | Current state analysis, migration strategy, resource allocation |
| Environment Setup | 3 weeks | Week 3 | Week 5 | Unity Catalog workspace setup, security configuration |
| Pilot Migration | 4 weeks | Week 6 | Week 9 | Migrate 2-3 critical datasets, validate processes |
| Bulk Migration Phase 1 | 6 weeks | Week 10 | Week 15 | Migrate 60% of datasets and workflows |
| Bulk Migration Phase 2 | 6 weeks | Week 16 | Week 21 | Complete remaining migrations |
| Testing & Validation | 3 weeks | Week 22 | Week 24 | End-to-end testing, performance validation |
| Go-Live & Support | 2 weeks | Week 25 | Week 26 | Production cutover, hypercare support |

## Resource Requirements

| Role | FTE | Duration | Responsibilities |
|------|-----|----------|------------------|
| Migration L

2025/09/16 09:37:42 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content="[[ ## re...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])"



  RISK ASSESSMENT:
**CRITICAL RISKS:**

**1. Regulatory Compliance & Data Sovereignty (HIGH RISK - CRITICAL IMPACT)**
- **Risk**: Energy sector regulations (GDPR, industry-specific compliance) may have specific data residency and governance requirements that Unity Catalog configuration might not initially meet
- **Likelihood**: Medium-High (energy companies face strict regulatory oversight)
- **Mitigation**: 
  - Conduct early regulatory compliance assessment with legal/compliance teams
  - Engage Databricks compliance specialists for energy sector requirements
  - Implement data classification and lineage tracking from day one
  - Plan for regulatory audit trails and data retention policies

**2. Business Continuity During Migration (HIGH RISK - CRITICAL IMPACT)**
- **Risk**: Centrica's energy operations likely run 24/7 with real-time data needs; any disruption could impact energy supply or trading operations
- **Likelihood**: Medium
- **Mitigation**:
  - Implement zero-downtime migr

{'project_plan': '## Migration Timeline\n\n| Phase | Duration | Start Week | End Week | Key Deliverables |\n|-------|----------|------------|----------|------------------|\n| Assessment & Planning | 4 weeks | Week 1 | Week 4 | Current state analysis, migration strategy, resource allocation |\n| Environment Setup | 3 weeks | Week 3 | Week 5 | Unity Catalog workspace setup, security configuration |\n| Pilot Migration | 4 weeks | Week 6 | Week 9 | Migrate 2-3 critical datasets, validate processes |\n| Bulk Migration Phase 1 | 6 weeks | Week 10 | Week 15 | Migrate 60% of datasets and workflows |\n| Bulk Migration Phase 2 | 6 weeks | Week 16 | Week 21 | Complete remaining migrations |\n| Testing & Validation | 3 weeks | Week 22 | Week 24 | End-to-end testing, performance validation |\n| Go-Live & Support | 2 weeks | Week 25 | Week 26 | Production cutover, hypercare support |\n\n## Resource Requirements\n\n| Role | FTE | Duration | Responsibilities |\n|------|-----|----------|---------------

[Trace(trace_id=tr-7c461b1c9caa328c0899f2d88ac3e25f), Trace(trace_id=tr-92b50ce91a51d9867e08cbd2ac0188c8), Trace(trace_id=tr-d7fd6f7089bb781b15e27b7a18ce17af)]

In [0]:
# Deploy Agent using DSPy MLflow Integration
import mlflow
import mlflow.dspy
from mlflow.models.resources import DatabricksVectorSearchIndex, DatabricksServingEndpoint

# Set registry URI for Unity Catalog
mlflow.set_registry_uri("databricks-uc")

# Input example for the model
input_example = {
    "messages": [
        {
            "role": "user",
            "content": "How many team members are there for the Databricks migration? Category: Resource. Context: We have 5 team members."
        }
    ]
}

# Create a DSPy-compatible wrapper for the agent
class DeliveryPlanningAgentDSPy(dspy.Module):
    """DSPy-compatible wrapper for the Delivery Planning Agent."""
    
    def __init__(self):
        super().__init__()
        self.agent = None
        
    def forward(self, question, category="General", context=""):
        """Forward method for DSPy."""
        try:
            if not self.agent:
                # Initialize the agent if not already done
                self.agent = DeliveryPlanningAgent(
                    vector_search_endpoint=VECTOR_SEARCH_ENDPOINT_NAME,
                    vector_search_index=VECTOR_SEARCH_INDEX_NAME
                )
            
            result = self.agent.answer_question(question, context, category)
            return dspy.Prediction(answer=result["answer"])
        except Exception as e:
            return dspy.Prediction(answer=f"Error: {str(e)}")

# Create the DSPy model instance
dspy_agent = DeliveryPlanningAgentDSPy()

# Log the agent using DSPy MLflow integration
with mlflow.start_run() as run:
    logged_agent_info = mlflow.dspy.log_model(
        dspy_model=dspy_agent,
        artifact_path="delivery-planning-agent",
        input_example=input_example,
        task="llm/v1/chat",
        model_config={
            "vector_search_endpoint": VECTOR_SEARCH_ENDPOINT_NAME,
            "vector_search_index": VECTOR_SEARCH_INDEX_NAME
        },
        resources=[
            DatabricksVectorSearchIndex(index_name=VECTOR_SEARCH_INDEX_NAME),
            DatabricksServingEndpoint(endpoint_name="databricks-claude-sonnet-4")
        ]
    )
    
    print(f" Agent logged to MLflow: {logged_agent_info.model_uri}")

# Register the agent in Unity Catalog
model_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.delivery-planning-agent"
uc_model_info = mlflow.register_model(model_uri=logged_agent_info.model_uri, name=model_name)

print(f" Agent registered in Unity Catalog: {uc_model_info.name}")
print(f"   Version: {uc_model_info.version}")

print(f" Agent ready for deployment!")
print(f" Model URI: {logged_agent_info.model_uri}")
print(f"  Registered as: {uc_model_info.name}")
print(f" Next step: Deploy via Databricks Model Serving UI or API")
print(f" Use the model URI to create a serving endpoint")


2025/09/16 09:37:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
🔗 View Logged Model at: https://adb-984752964297111.11.azuredatabricks.net/ml/experiments/3013595711630001/models/m-e7aa2164dc0b41a487fedc83e20fd365?o=984752964297111
2025/09/16 09:38:08 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-e7aa2164dc0b41a487fedc83e20fd365
2025/09/16 09:38:08 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.



 Processing Answer:
Q: [{'role': 'user', 'content': 'How many team members are there for the Databricks migration? Category: Resource. Context: We have 5 team members.'}]
A: 
[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


 Agent logged to MLflow: models:/m-e7aa2164dc0b41a487fedc83e20fd365


---------------------------------------------------------------------------
RestException                             Traceback (most recent call last)
File <command-8479387074310181>, line 66
     64 # Register the agent in Unity Catalog
     65 model_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.delivery-planning-agent"
---> 66 uc_model_info = mlflow.register_model(model_uri=logged_agent_info.model_uri, name=model_name)
     68 print(f" Agent registered in Unity Catalog: {uc_model_info.name}")
     69 print(f"   Version: {uc_model_info.version}")

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-955dad53-167e-4691-b82c-fb179c6805b2/lib/python3.12/site-packages/mlflow/tracking/_model_registry/fluent.py:129, in register_model(model_uri, name, await_registration_for, tags, env_pack)
     61 def register_model(
     62     model_uri,
     63     name,
   (...)
     67     env_pack: EnvPackType | None = None,
     68 ) -> ModelVersion:
     69     """Create a new model version in model registry fo